<a href="https://colab.research.google.com/github/ruebot/notebooks/blob/main/digfemnet/uwaterooconfessions_misogyny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gspread
import pandas as pd

from google.auth import default
from google.colab import auth
from transformers import pipeline

In [2]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

worksheet = gc.open('uwaterlooconfessions').sheet1

rows = worksheet.get_all_values()

df = pd.DataFrame(rows[1:], columns=rows[0])

df.head()


,Filename,Post ID,Post date,OCR'd Confession Text
0,uwaterlooconfessions-3483170083883540949-2024-...,3483170083883540949,2024-10-20_17-44-12-UTC_3,CONFESSION #1631\n(Part 3) INSTEAD OF PUTTING ...
1,uwaterlooconfessions-3483170083883540949-2024-...,3483170083883540949,2024-10-20_17-44-12-UTC_2,CONFESSION #1631\n(Part 2) I saw THIS HAPPEN T...
2,uwaterlooconfessions-3483170083883540949-2024-...,3483170083883540949,2024-10-20_17-44-12-UTC_1,CONFESSION #163]\n(Part x) ’ve noticep THat In...
3,uwaterlooconfessions-3482408221220915554-2024-...,3482408221220915554,2024-10-19_16-30-00-UTC_2,Xo 7 +5ke)\ E aLeele)\n(ParT 2) VEGANISM IS NO...
4,uwaterlooconfessions-3482408221220915554-2024-...,3482408221220915554,2024-10-19_16-30-00-UTC_1,CONFESSION #1630\n(Part 1) PEOPLE IX THE ENVIR...


In [3]:
pipe = pipeline("text-classification", model="annahaz/xlm-roberta-base-finetuned-misogyny-sexism")

def classify_text(text):
    result = pipe(text)[0]
    return result['label'], result['score']

df[['label', 'score']] = df["OCR'd Confession Text"].apply(lambda x: pd.Series(classify_text(x)))

df

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


,Filename,Post ID,Post date,OCR'd Confession Text,label,score
0,uwaterlooconfessions-3483170083883540949-2024-...,3483170083883540949,2024-10-20_17-44-12-UTC_3,CONFESSION #1631\n(Part 3) INSTEAD OF PUTTING ...,0,0.999199
1,uwaterlooconfessions-3483170083883540949-2024-...,3483170083883540949,2024-10-20_17-44-12-UTC_2,CONFESSION #1631\n(Part 2) I saw THIS HAPPEN T...,0,0.983896
2,uwaterlooconfessions-3483170083883540949-2024-...,3483170083883540949,2024-10-20_17-44-12-UTC_1,CONFESSION #163]\n(Part x) ’ve noticep THat In...,0,0.999790
3,uwaterlooconfessions-3482408221220915554-2024-...,3482408221220915554,2024-10-19_16-30-00-UTC_2,Xo 7 +5ke)\ E aLeele)\n(ParT 2) VEGANISM IS NO...,0,0.998131
4,uwaterlooconfessions-3482408221220915554-2024-...,3482408221220915554,2024-10-19_16-30-00-UTC_1,CONFESSION #1630\n(Part 1) PEOPLE IX THE ENVIR...,1,0.994304
...,...,...,...,...,...,...
1711,uwaterlooconfessions-1900411762068740730-2018-...,1900411762068740730,2018-10-28_22-45-57-UTC,CONFESSION #2\ni Aa aNnae MET ANYONE ON\nMY FL...,0,0.999372
1712,uwaterlooconfessions-1900353159807012064-2018-...,1900353159807012064,2018-10-28_20-49-31-UTC,CONFESSION #1\nTo THE WHITE GUY WITH\nBLACK HA...,1,0.953275
1713,uwaterlooconfessions-3483857736283002698-2024-...,3483857736283002698,2024-10-21_16-30-00-UTC,CONFESSION #1632\n‘Olsen OF THE SCHULICH LEADE...,0,0.999433
1714,uwaterlooconfessions-3484582360952281047-2024-...,3484582360952281047,2024-10-22_16-30-00-UTC,CONFESSION #1633\nIs IT BAD TO TRY AND DATE AN...,1,0.992034
